<a href="https://colab.research.google.com/github/TristanFaine/Master_2_MLVC_Recognize_Handwritten_Equation/blob/main/code_template_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task

We were given InkML files, which contain metadata and a list of strokes:  
[(0,0),(1;0)]...  
These are on-line handwritten mathematical expressions, we'll try to recognize them by using a LG(Labelled Graph) as intermediate output.
The sequence of actions will look like this for now:  
1) Get each stroke  
2) Convert each stroke to symbols (all possible combinations for now we'll optimize later)  
Two strategies available: Consider only consecutive strokes, or allow X(1) number of "time jumps" for each possible symbol.  
2.5) Make a classifier to ignore "junk" stroke combinations, such as a symbol created with every single stroke at the same time.  
3) Classify each constructed symbol with a neural network.

We'll handle spatial relations later  
We'll also handle the final decision later, but we can add a grammar or language model at that point, 

#Environment setup

Please import each file from the git repository to see how they work.  
TODO: link from the git repo, should be feasible.

We will first try out each script then explain what they're supposed to do and how to interpret their output.

##Useful stuff
### Displaying hypotheses/images from strokes


In [ ]:
if __name__ == '__main__':
    from convertInkmlToImg import parse_inkml, genImgFromLGHypotheses, convert_to_imgs
    from segmenter import generateHypSeg
    # render one image for every symbol in inkml file using GT segmentation
    traces = parse_inkml(sys.argv[1])
    with open(sys.argv[2], 'r') as lg_f: 
        lg = lg_f.read()
    imgs = genImgFromLGHypotheses(traces, lg, 64)
    
    # Add padding and gaussian blur yourself
    # Plot returned images
    import matplotlib.pyplot as plt
    plt.figure()
    N = imgs.shape[0]
    for i in range(N):
        ax = plt.subplot(np.ceil(np.sqrt(N)), np.ceil(np.sqrt(N)), i+1)
        plt.imshow(imgs[i], origin='upper', alpha = 1)
        ax.set_xticklabels([]);ax.set_yticklabels([])
    plt.show()
    plt.figure()
    imgs = np.zeros((len(traces), 64, 64))
    for u, (i, trace) in enumerate(traces.items()):
        
        imgs[u] = convert_to_imgs(get_traces_data(traces, [i]), 64)
    hypothesis = generateHypSeg(len(imgs))
    N = len(hypothesis)
    for i in range(N):
        ax = plt.subplot(np.ceil(np.sqrt(N)), np.ceil(np.sqrt(N)), i+1)
        # imgs[hypothesis[i]].sum()
        plt.imshow((255 - imgs[hypothesis[i]]).sum(0).clip(0,255), origin='upper', alpha = 1)
        ax.set_xticklabels([]);ax.set_yticklabels([])
    plt.show()
    print('end')

## Data to train our classifiers

[This](https://uncloud.univ-nantes.fr/index.php/s/OdAtErZgxKGjsNy) contains a bunch of symbols (in datasymbol_iso/) and expressions (in FullExpressions/) to help train our future classifiers.

#WARNING: training size
there are a LOT of images in the CHROHME dataset, only use a subset to train our classifiers otherwise it'll take too long.

##segmenter.py explanation

In [ ]:
!chmod 777 ./segmenter.py

In [ ]:
!python3 segmenter.py -i formulaire001-equation001.inkml -o bidule.lg

In [ ]:
!python3 segmenter.py -i formulaire001-equation001.inkml

O,hyp0,*,1.0,0
O,hyp1,*,1.0,1
O,hyp2,*,1.0,2
O,hyp3,*,1.0,3
O,hyp4,*,1.0,4
O,hyp5,*,1.0,0,1
O,hyp6,*,1.0,1,2
O,hyp7,*,1.0,2,3
O,hyp8,*,1.0,3,4
O,hyp9,*,1.0,0,1,2
O,hyp10,*,1.0,1,2,3
O,hyp11,*,1.0,2,3,4
O,hyp12,*,1.0,0,1,2,3
O,hyp13,*,1.0,1,2,3,4



Each line of this output corresponds to a hypothesis, we can ignore the first four values since they're dummy values indicating a symbol type thing, maybe the position, then the symbol with index, then the symbol without index, then the supposed confidence of our future model(s).. probably, then the strokes used to make that symbol.

For now, we simply list every possible symbol, using every possible stroke in sequence, so if we have 4 strokes, we can make 13 possible symbols.  

segmentSelect.py can be used to remove some hypotheses that don't make sense, for instance, the symbol that could be constructed with stroke [0,1,2,3,4] is highly unlikely to appear, so after training our classifier, we'd remove it.  
Ground truth can be acquired from the lg files of the texts, since we have the strokes used as the 5th...i values.

##segmentSelect.py explanation

In [ ]:
!chmod 777 ./segmentSelect.py

In [ ]:
!python3 segmentSelect.py -o bidule2.lg formulaire001-equation001.inkml bidule.lg 

In [ ]:
!python3 segmentSelect.py formulaire001-equation001.inkml bidule.lg 

O,hyp1,*,0.2115377460481933,1
O,hyp2,*,0.7589762933778768,2
O,hyp3,*,0.9465286160422606,3
O,hyp4,*,0.6472040009605318,4
O,hyp5,*,0.5317501548061437,0,1
O,hyp6,*,0.869769890079878,1,2
O,hyp7,*,0.7930116985818777,2,3
O,hyp8,*,0.8592390037323104,3,4
O,hyp10,*,0.8535922279194446,1,2,3
O,hyp11,*,0.30968321364070184,2,3,4
O,hyp12,*,0.2548672487970586,0,1,2,3
O,hyp13,*,0.8645322026100054,1,2,3,4



This script takes as input the initial inkml file, alongside the "prototype" lg file: We combine the stroke combinations alongside the inkml data to generate images, then we check whether these make sense as a symbol, no matter the context.

For now this simply gives a fully random probability that the images are valid symbols.

Need to change two things in there, first adding a classifier to check the validity of supposed symbols,  
then change the threshold based on empirical evidence or what I feel like at the time.

##segmentReco.py explanation

In [ ]:
!chmod 777 ./symbolReco.py

In [ ]:
!python3 symbolReco.py formulaire001-equation001.inkml bidule2.lg

O,hyp0,1,0.05955472045107981,0
O,hyp1,h,0.052429356330855326,1
O,hyp1,1,0.05051970752475116,1
O,hyp1,div,0.0518316643671452,1
O,hyp2,h,0.05048942606554477,2
O,hyp2,t,0.06074056070259754,2
O,hyp3,k,0.057856038434335566,3
O,hyp3,-,0.07226818345595058,3
O,hyp3,gt,0.057580774760517844,3
O,hyp4,u,0.05575209296535842,4
O,hyp4,),0.05543832753567732,4
O,hyp5,u,0.059863605019343916,0,1
O,hyp5,{,0.05925994011792812,0,1
O,hyp5,sum,0.06377157901330438,0,1
O,hyp6,j,0.05926945552690245,1,2
O,hyp7,o,0.050373341841529105,2,3
O,hyp7,x,0.06696417617840533,2,3
O,hyp7,B,0.05192586201868759,2,3
O,hyp7,0,0.055163562984986945,2,3
O,hyp8,i,0.053063307028174286,3,4
O,hyp8,k,0.0641493953924078,3,4
O,hyp8,l,0.057927585701751626,3,4
O,hyp8,C,0.052309317598633866,3,4
O,hyp9,t,0.05971415659432129,0,1,2
O,hyp9,sum,0.05125243846652762,0,1,2
O,hyp9,gt,0.05427977693390457,0,1,2
O,hyp10,r,0.051049265478135285,1,2,3
O,hyp10,(,0.059080584931592246,1,2,3
O,hyp11,s,0.052391352664092374,2,3,4

